In [29]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.sequence import pad_sequences

import os

In [30]:
imdb_train, ds_info = tfds.load(name="imdb_reviews",
                                split="train",
                                with_info=True, as_supervised=True)

imdb_test = tfds.load(name="imdb_reviews", split="test", as_supervised=True)

INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load dataset info from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
INFO:absl:Reusing dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split train, from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load dataset info from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
INFO:absl:Reusing dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split test, from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0


### Keras Tokenizer

In [3]:
X_train = list(map(lambda x: x[0].numpy().decode('utf-8'), imdb_train))
y_train = list(map(lambda x: x[1], imdb_train))

In [ ]:
%%time
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                                  lower=True,
                                                  split=" ",)
tokenizer.fit_on_texts(X_train)

X_encoded = tokenizer.texts_to_sequences(X_train)
X_encoded = pad_sequences(X_encoded, padding="post", maxlen=150) 

Wall time: 9.88 s


### TFDS Tokenizer

In [31]:
tokenizer_tfds = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()
MAX_TOKENS = 0
for example, label in imdb_train:
    some_tokens = tokenizer_tfds.tokenize(example.numpy())
    if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
    vocabulary_set.update(some_tokens)
    
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, lowercase=True, tokenizer=tokenizer_tfds)

vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKENS)

93931 2525


In [32]:
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post',maxlen=150)
    return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
    
    encoded = tf.py_function(encode_pad_transform,
                             inp=[sample],
                             Tout=(tf.int64))
    
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label 

In [33]:
encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

### Loading pre-trained GloVe embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [34]:
from google.colab import drive

drive.mount('drive')
glove_path = 'drive/MyDrive/GloVe/glove.6B.50d.txt'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [35]:
dict_w2v = {}
with open(glove_path, "rb") as file:
    for line in file:
        tokens = line.split()
        word = tokens[0].decode('utf-8')
        vector = np.array(tokens[1:], dtype=np.float32)
        
        if vector.shape[0] == 50:
            dict_w2v[word] = vector
        else:
            print("There was an issue with " + word)
            
# let's check the vocabulary size
print("Dictionary Size: ", len(dict_w2v))

Dictionary Size:  400000


In [36]:
vocab_size = imdb_encoder.vocab_size
embedding_dim = 50
embedding_matrix = np.zeros((vocab_size, embedding_dim))

### Tfds

In [37]:
unk_cnt = 0
unk_set = set()
for word in imdb_encoder.tokens:
    embedding_vector = dict_w2v.get(word)
    
    if embedding_vector is not None:
        tkn_id = imdb_encoder.encode(word)[0]
        embedding_matrix[tkn_id] = embedding_vector
    else:
        unk_cnt += 1
        unk_set.add(word)
# Print how many weren't found
print("Total unknown words: ", unk_cnt)

Total unknown words:  14553


### Keras

In [ ]:
unk_cnt = 0
unk_set = set()
for word in list(tokenizer.word_index.keys()):
    embedding_vector = dict_w2v.get(word)
    
    if embedding_vector is not None:
        tkn_id = tokenizer.word_index[word]
        embedding_matrix[tkn_id] = embedding_vector
    else:
        unk_cnt += 1
        unk_set.add(word)
# Print how many weren't found
print("Total unknown words: ", unk_cnt)

Total unknown words:  28423


### Build model

In [38]:
vocab_size = imdb_encoder.vocab_size # len(chars)
# Number of RNN units
rnn_units = 64
#batch size
BATCH_SIZE=100

In [39]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size, train_emb=False):
    model = tf.keras.Sequential([
        Embedding(vocab_size, embedding_dim, mask_zero=True,
                  weights=[embedding_matrix], trainable=train_emb),
        Bidirectional(LSTM(rnn_units, return_sequences=True,
                           dropout=0.5)),
        Bidirectional(LSTM(rnn_units, dropout=0.25)),
        Dense(1, activation='sigmoid')
    ])
    return model

In [40]:
model_fe = build_model_bilstm(
    vocab_size = vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model_fe.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 50)          4696550   
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 128)         58880     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 4,854,375
Trainable params: 157,825
Non-trainable params: 4,696,550
_________________________________________________________________


In [41]:
model_fe.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])

In [42]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)
model_fe.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 33s 82ms/step - loss: 0.6347 - accuracy: 0.6282 - precision: 0.6288 - recall: 0.6348
Epoch 2/10
250/250 [==============================] - 22s 90ms/step - loss: 0.5586 - accuracy: 0.7157 - precision: 0.7223 - recall: 0.7025
Epoch 3/10
250/250 [==============================] - 23s 90ms/step - loss: 0.5194 - accuracy: 0.7432 - precision: 0.7470 - recall: 0.7379
Epoch 4/10
250/250 [==============================] - 22s 90ms/step - loss: 0.4896 - accuracy: 0.7647 - precision: 0.7644 - recall: 0.7665
Epoch 5/10
250/250 [==============================] - 23s 90ms/step - loss: 0.5145 - accuracy: 0.7462 - precision: 0.7491 - recall: 0.7424
Epoch 6/10
250/250 [==============================] - 22s 90ms/step - loss: 0.4608 - accuracy: 0.7800 - precision: 0.7829 - recall: 0.7757
Epoch 7/10
250/250 [==============================] - 23s 91ms/step - loss: 0.4534 - accuracy: 0.7883 - precision: 0.7968 - recall: 0.7752
Epoch 8/10
250/250 [=======

### Fine-tuning

In [43]:
model_ft = build_model_bilstm(
 vocab_size=vocab_size,
 embedding_dim=embedding_dim,
 rnn_units=rnn_units,
 batch_size=BATCH_SIZE,
 train_emb=True)
model_ft.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 50)          4696550   
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 128)         58880     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 4,854,375
Trainable params: 4,854,375
Non-trainable params: 0
_________________________________________________________________


In [44]:
model_ft.compile(loss='binary_crossentropy',
 optimizer='adam',
 metrics=['accuracy', 'Precision', 'Recall'])
model_ft.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 44s 125ms/step - loss: 0.6129 - accuracy: 0.6483 - precision: 0.6497 - recall: 0.6520
Epoch 2/10
250/250 [==============================] - 33s 131ms/step - loss: 0.4159 - accuracy: 0.8108 - precision: 0.8106 - recall: 0.8120
Epoch 3/10
250/250 [==============================] - 33s 131ms/step - loss: 0.3367 - accuracy: 0.8522 - precision: 0.8549 - recall: 0.8491
Epoch 4/10
250/250 [==============================] - 33s 130ms/step - loss: 0.2909 - accuracy: 0.8764 - precision: 0.8749 - recall: 0.8789
Epoch 5/10
250/250 [==============================] - 33s 131ms/step - loss: 0.2436 - accuracy: 0.9008 - precision: 0.9008 - recall: 0.9011
Epoch 6/10
250/250 [==============================] - 33s 131ms/step - loss: 0.2209 - accuracy: 0.9113 - precision: 0.9097 - recall: 0.9135
Epoch 7/10
250/250 [==============================] - 33s 131ms/step - loss: 0.1858 - accuracy: 0.9259 - precision: 0.9270 - recall: 0.9250
Epoch 8/10
250/250 [

### BERT

In [ ]:
!pip install transformers==3.0.2

In [ ]:
from transformers import BertTokenizer
bert_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_name,
                                          add_special_tokens=True,
                                          do_lower_case=False,
                                          max_length=150,
                                          pad_to_max_length=True)

In [ ]:
def bert_encoder(review):
 txt = review.numpy().decode('utf-8')
 encoded = tokenizer.encode_plus(txt, add_special_tokens=True,
                                 max_length=150,
                                 pad_to_max_length=True,
                                 return_attention_mask=True,
                                 truncation=True,
                                 return_token_type_ids=True)
 
 return encoded['input_ids'], encoded['token_type_ids'], encoded['attention_mask']

In [ ]:
bert_train = [bert_encoder(r) for r, l in imdb_train]
bert_lbl = [l for r, l in imdb_train]
bert_train = np.array(bert_train)
bert_lbl = tf.keras.utils.to_categorical(bert_lbl, num_classes=2)